In [1]:
!pip install pyspark findspark


In [2]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession


sc = SparkContext.getOrCreate()
ss = SparkSession(sc).builder.config("spark.driver.memory", "6g").getOrCreate()

import pyspark.sql.functions as f
#import pyspark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
weather_df = ss.read.option('header', True).csv("aligned data/weather-ta2.csv").limit(20000)
weather_df.show()

+---+-----------+----------+-------+-------+
|_c0| station_id|      date|tmp_val|dew_val|
+---+-----------+----------+-------+-------+
|  0|01016099999|2016-01-01|      4|     -7|
|  1|01016099999|2016-01-02|      6|     -1|
|  2|01016099999|2016-01-03|      0|    -21|
|  3|01016099999|2016-01-04|      0|     -9|
|  4|01016099999|2016-01-05|    -53|    -61|
|  5|01016099999|2016-01-06|   -104|   -112|
|  6|01016099999|2016-01-07|    -62|    -81|
|  7|01016099999|2016-01-08|   -102|   -151|
|  8|01016099999|2016-01-09|    -72|    -88|
|  9|01016099999|2016-01-10|    -69|    -97|
| 10|01016099999|2016-01-11|    -93|   -122|
| 11|01016099999|2016-01-12|   -126|   -139|
| 12|01016099999|2016-01-13|    -86|   -107|
| 13|01016099999|2016-01-14|    -71|    -96|
| 14|01016099999|2016-01-15|    -61|    -78|
| 15|01016099999|2016-01-16|    -62|    -79|
| 16|01016099999|2016-01-17|    -27|    -32|
| 17|01016099999|2016-01-18|    -30|    -39|
| 18|01016099999|2016-01-19|    -42|    -54|
| 19|01016

In [4]:
weather_df = sc.parallelize(weather_df)

TypeError: cannot pickle '_thread.RLock' object

In [5]:
stocks_df=ss.read.option('header', True).csv('/content/drive/MyDrive/BigDataManagement/final data/stocks-ta2.csv')
stocks_df.show()

+---+--------------------+----------+-----+
|_c0|          stock_name|      date|price|
+---+--------------------+----------+-----+
|  0|1004.Asien--Austr...|2016-01-01| 57.5|
|  1|1004.Asien--Austr...|2016-01-02| 57.5|
|  2|1004.Asien--Austr...|2016-01-03| 57.5|
|  3|1004.Asien--Austr...|2016-01-04| 57.8|
|  4|1004.Asien--Austr...|2016-01-05|57.17|
|  5|1004.Asien--Austr...|2016-01-06|57.65|
|  6|1004.Asien--Austr...|2016-01-07| 58.4|
|  7|1004.Asien--Austr...|2016-01-08|56.44|
|  8|1004.Asien--Austr...|2016-01-09|56.44|
|  9|1004.Asien--Austr...|2016-01-10|56.44|
| 10|1004.Asien--Austr...|2016-01-11| 56.0|
| 11|1004.Asien--Austr...|2016-01-12|56.72|
| 12|1004.Asien--Austr...|2016-01-13|56.11|
| 13|1004.Asien--Austr...|2016-01-14|55.08|
| 14|1004.Asien--Austr...|2016-01-15|56.97|
| 15|1004.Asien--Austr...|2016-01-16|56.97|
| 16|1004.Asien--Austr...|2016-01-17|56.97|
| 17|1004.Asien--Austr...|2016-01-18|54.69|
| 18|1004.Asien--Austr...|2016-01-19|56.33|
| 19|1004.Asien--Austr...|2016-0

In [6]:
weather_df.registerTempTable("df")
 
mylist = ss.sql("""
SELECT DISTINCT  u1.station_id,
    u2.station_id
FROM df as u1 JOIN df as u2 ON u1.station_id> u2.station_id;
""").collect()
 


/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [7]:
weather_df.columns

['_c0', 'station_id', 'date', 'tmp_val', 'dew_val']

In [8]:
weather_df = weather_df.select('station_id', 'date', 'tmp_val')

In [12]:
i, j = mylist[0]

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType

schema = weather_df.filter(weather_df.station_id == i).alias('a').join(weather_df.filter(weather_df.station_id == j)\
                                         .withColumnRenamed('station_id', 'station_id_2')\
                                         .withColumnRenamed('date', 'date_2')\
                                         .withColumnRenamed('tmp_val', 'tmp_val_2').alias('b'),
                                            f.col("a.date") == f.col("b.date_2"), 'inner').schema

df = ss.createDataFrame([], schema)

for a, b in mylist:
    df = df.union(weather_df.filter(weather_df.station_id == a).alias('a').join(weather_df.filter(weather_df.station_id == b)\
                                         .withColumnRenamed('station_id', 'station_id_2')\
                                         .withColumnRenamed('date', 'date_2')\
                                         .withColumnRenamed('tmp_val', 'tmp_val_2').alias('b'),
                                            f.col("a.date") == f.col("b.date_2"), 'inner'))

In [15]:
df.count()

190000

In [16]:
df.show()

+-----------+----------+-------+------------+----------+---------+
| station_id|      date|tmp_val|station_id_2|    date_2|tmp_val_2|
+-----------+----------+-------+------------+----------+---------+
|02520099999|2016-01-01|      6| 01016099999|2016-01-01|        4|
|02520099999|2016-01-02|      4| 01016099999|2016-01-02|        6|
|02520099999|2016-01-03|    -38| 01016099999|2016-01-03|        0|
|02520099999|2016-01-04|    -58| 01016099999|2016-01-04|        0|
|02520099999|2016-01-05|    -65| 01016099999|2016-01-05|      -53|
|02520099999|2016-01-06|    -91| 01016099999|2016-01-06|     -104|
|02520099999|2016-01-07|   -130| 01016099999|2016-01-07|      -62|
|02520099999|2016-01-08|    -47| 01016099999|2016-01-08|     -102|
|02520099999|2016-01-09|    -49| 01016099999|2016-01-09|      -72|
|02520099999|2016-01-10|    -25| 01016099999|2016-01-10|      -69|
|02520099999|2016-01-11|     12| 01016099999|2016-01-11|      -93|
|02520099999|2016-01-12|      5| 01016099999|2016-01-12|     -

In [17]:
df = df.select('station_id', 'station_id_2', 'date', 'tmp_val', 'tmp_val_2')

In [18]:
from pyspark.sql.types import StringType, FloatType

def max(a,b):
  if float(a) > float(b):
    return float(a)
  else:
    return float(b)

def min(a,b):
  if float(a) > float(b):
    return float(b)
  else:
    return float(a)

def avg(a,b):
  return (float(a) + float(b))/2

max_udf = f.udf(max, FloatType())
min_udf = f.udf(min, FloatType())
avg_udf = f.udf(avg, FloatType())

In [19]:
df = df.withColumn('min', min_udf('tmp_val', 'tmp_val_2'))\
  .withColumn('avg', avg_udf('tmp_val', 'tmp_val_2'))\
  .withColumn('max', max_udf('tmp_val', 'tmp_val_2'))

In [20]:
df.show()

+-----------+------------+----------+-------+---------+------+------+-----+
| station_id|station_id_2|      date|tmp_val|tmp_val_2|   min|   avg|  max|
+-----------+------------+----------+-------+---------+------+------+-----+
|02520099999| 01016099999|2016-01-01|      6|        4|   4.0|   5.0|  6.0|
|02520099999| 01016099999|2016-01-02|      4|        6|   4.0|   5.0|  6.0|
|02520099999| 01016099999|2016-01-03|    -38|        0| -38.0| -19.0|  0.0|
|02520099999| 01016099999|2016-01-04|    -58|        0| -58.0| -29.0|  0.0|
|02520099999| 01016099999|2016-01-05|    -65|      -53| -65.0| -59.0|-53.0|
|02520099999| 01016099999|2016-01-06|    -91|     -104|-104.0| -97.5|-91.0|
|02520099999| 01016099999|2016-01-07|   -130|      -62|-130.0| -96.0|-62.0|
|02520099999| 01016099999|2016-01-08|    -47|     -102|-102.0| -74.5|-47.0|
|02520099999| 01016099999|2016-01-09|    -49|      -72| -72.0| -60.5|-49.0|
|02520099999| 01016099999|2016-01-10|    -25|      -69| -69.0| -47.0|-25.0|
|02520099999

In [21]:
def unqiue_combination(a, b):
  col = [a, b]
  col.sort()
  return '_'.join(col)

my_udf = f.udf(unqiue_combination, StringType())

In [22]:
df = df.withColumn('combo_id', my_udf('station_id', 'station_id_2')).select('combo_id', 'date', 'min', 'avg', 'max')

In [23]:
df.show()

+--------------------+----------+------+------+-----+
|            combo_id|      date|   min|   avg|  max|
+--------------------+----------+------+------+-----+
|01016099999_02520...|2016-01-01|   4.0|   5.0|  6.0|
|01016099999_02520...|2016-01-02|   4.0|   5.0|  6.0|
|01016099999_02520...|2016-01-03| -38.0| -19.0|  0.0|
|01016099999_02520...|2016-01-04| -58.0| -29.0|  0.0|
|01016099999_02520...|2016-01-05| -65.0| -59.0|-53.0|
|01016099999_02520...|2016-01-06|-104.0| -97.5|-91.0|
|01016099999_02520...|2016-01-07|-130.0| -96.0|-62.0|
|01016099999_02520...|2016-01-08|-102.0| -74.5|-47.0|
|01016099999_02520...|2016-01-09| -72.0| -60.5|-49.0|
|01016099999_02520...|2016-01-10| -69.0| -47.0|-25.0|
|01016099999_02520...|2016-01-11| -93.0| -40.5| 12.0|
|01016099999_02520...|2016-01-12|-126.0| -60.5|  5.0|
|01016099999_02520...|2016-01-13| -86.0| -56.5|-27.0|
|01016099999_02520...|2016-01-14| -73.0| -72.0|-71.0|
|01016099999_02520...|2016-01-15|-142.0|-101.5|-61.0|
|01016099999_02520...|2016-0

------

In [24]:
stocks_df=ss.read.option('header', True).csv('/content/drive/MyDrive/BigDataManagement/final data/stocks-ta2.csv').limit(20000)
stocks_df.show()

+---+--------------------+----------+-----+
|_c0|          stock_name|      date|price|
+---+--------------------+----------+-----+
|  0|1004.Asien--Austr...|2016-01-01| 57.5|
|  1|1004.Asien--Austr...|2016-01-02| 57.5|
|  2|1004.Asien--Austr...|2016-01-03| 57.5|
|  3|1004.Asien--Austr...|2016-01-04| 57.8|
|  4|1004.Asien--Austr...|2016-01-05|57.17|
|  5|1004.Asien--Austr...|2016-01-06|57.65|
|  6|1004.Asien--Austr...|2016-01-07| 58.4|
|  7|1004.Asien--Austr...|2016-01-08|56.44|
|  8|1004.Asien--Austr...|2016-01-09|56.44|
|  9|1004.Asien--Austr...|2016-01-10|56.44|
| 10|1004.Asien--Austr...|2016-01-11| 56.0|
| 11|1004.Asien--Austr...|2016-01-12|56.72|
| 12|1004.Asien--Austr...|2016-01-13|56.11|
| 13|1004.Asien--Austr...|2016-01-14|55.08|
| 14|1004.Asien--Austr...|2016-01-15|56.97|
| 15|1004.Asien--Austr...|2016-01-16|56.97|
| 16|1004.Asien--Austr...|2016-01-17|56.97|
| 17|1004.Asien--Austr...|2016-01-18|54.69|
| 18|1004.Asien--Austr...|2016-01-19|56.33|
| 19|1004.Asien--Austr...|2016-0

In [25]:
stocks_df = stocks_df.withColumn('price', f.col('price').cast(FloatType()))

In [26]:
from pyspark.sql.types import FloatType, DoubleType

big_df = stocks_df.alias('a').join(df.alias('b'), f.col('a.date') == f.col('b.date'), 'inner')
#big_df = big_df.withColumn("cosine_sim", f.lit(1.0).cast("float"))
big_df = big_df.select('a.date', 'stock_name', 'combo_id', 'price', 'avg', 'min', 'max', 'cosine_sim')

In [27]:
list(big_df.schema)

[StructField(date,StringType,true),
 StructField(stock_name,StringType,true),
 StructField(combo_id,StringType,true),
 StructField(price,FloatType,true),
 StructField(avg,FloatType,true),
 StructField(min,FloatType,true),
 StructField(max,FloatType,true),
 StructField(cosine_sim,FloatType,false)]

In [28]:
big_df.columns

['date', 'stock_name', 'combo_id', 'price', 'avg', 'min', 'max', 'cosine_sim']

In [29]:
big_df.count()

3800000

----

In [31]:
##big_df.show()

+----------+--------------------+--------------------+------+---+---+---+----------+
|      date|          stock_name|            combo_id| price|avg|min|max|cosine_sim|
+----------+--------------------+--------------------+------+---+---+---+----------+
|2016-01-01|11006.Asien--Aust...|01016099999_02520...| 318.0|5.0|4.0|6.0|       1.0|
|2016-01-01|10990.Asien--Aust...|01016099999_02520...| 520.0|5.0|4.0|6.0|       1.0|
|2016-01-01|10968.Asien--Aust...|01016099999_02520...| 960.5|5.0|4.0|6.0|       1.0|
|2016-01-01|10949.Asien--Aust...|01016099999_02520...|1289.0|5.0|4.0|6.0|       1.0|
|2016-01-01|10905.Asien--Aust...|01016099999_02520...|1328.0|5.0|4.0|6.0|       1.0|
|2016-01-01|10882.Asien--Aust...|01016099999_02520...|3110.0|5.0|4.0|6.0|       1.0|
|2016-01-01|1081.Asien--Austr...|01016099999_02520...| 0.415|5.0|4.0|6.0|       1.0|
|2016-01-01|10780.Asien--Aust...|01016099999_02520...|2220.0|5.0|4.0|6.0|       1.0|
|2016-01-01|1076.Asien--Austr...|01016099999_02520...|  6.15|5.0|

### Text below is a redundant code

from pyspark.sql.types import ArrayType, FloatType
import functools
def fudf(val):
    return functools.reduce(lambda x, y:x+y, val)

flattenUdf = f.udf(fudf, ArrayType(FloatType()))

In [43]:
bigdf_grouped = big_df.groupBy(["stock_name", 'combo_id']).agg(f.collect_list("price").alias('price'),
                                                               f.collect_list("avg").alias('avg'),
                                                               f.collect_list("min").alias('min'),
                                                               f.collect_list("max").alias('max'))
                                                          #.select("stock_name",'combo_id',
                                                          #        flattenUdf("min").alias("min"),
                                                          #        flattenUdf("max").alias("max"),
                                                          #        flattenUdf("avg").alias("avg"),
                                                          #        flattenUdf("price").alias("price"))


In [44]:
import numpy as np

def cosine(a, b):
  return float(np.dot(np.array(a), np.array(b)) / (np.linalg.norm(np.array(a)) * np.linalg.norm(np.array(b))))

In [45]:
cosine_udf = f.udf(cosine, FloatType())

----

In [46]:
df_final = bigdf_grouped.select('stock_name', 'combo_id', cosine_udf('price', 'max').alias('price_max_cosine'),
                                                          cosine_udf('price', 'min').alias('price_min_cosine'),
                                                          cosine_udf('price', 'avg').alias('price_avg_cosine'))

In [48]:
df_final.orderBy('price_avg_cosine', ascending=False).show()

+--------------------+--------------------+----------------+----------------+----------------+
|          stock_name|            combo_id|price_max_cosine|price_min_cosine|price_avg_cosine|
+--------------------+--------------------+----------------+----------------+----------------+
|10679.Asien--Aust...|06207099999_08524...|       0.9889081|       0.9046139|      0.96800274|
|10679.Asien--Aust...|03066099999_08524...|       0.9891048|       0.8502761|       0.9667747|
|10698.Asien--Aust...|06207099999_08524...|      0.98504055|       0.9026194|       0.9648411|
|10612.Asien--Aust...|06207099999_08524...|       0.9860405|      0.90060776|       0.9646363|
|10679.Asien--Aust...|03862099999_08524...|       0.9890492|       0.8536509|       0.9641612|
|1004.Asien--Austr...|06207099999_08524...|      0.97955555|      0.90912265|       0.9641316|
|10882.Asien--Aust...|06207099999_08524...|      0.98494434|      0.89949673|      0.96351963|
|10698.Asien--Aust...|03066099999_08524...|       

----

EVERYTHING BELOW IS REDUNDANT

----

In [ ]:
from pyspark.sql.functions import PandasUDFType, pandas_udf

In [ ]:
import numpy as np

# Names of columns 
a, b = "price", "avg" #('min', 'max', 'avg')
cosine_sim_col = "cosine_sim"

@pandas_udf(big_df.schema, PandasUDFType.GROUPED_MAP)
def cos_sim(df):
    df[cosine_sim_col] = float(np.dot(df[a], df[b]) / (np.linalg.norm(df[a]) * np.linalg.norm(df[b])))
    return df

In [ ]:
df_final = big_df.groupby(["stock_name", "combo_id"]).apply(cos_sim)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [ ]:
df_final.count()

3800000

In [ ]:
df_final.write.mode('overwrite').csv('/content/drive/MyDrive/BigDataManagement/final data/cosines.csv')

In [ ]:
df_final.groupBy(["stock_name", "combo_id"]).agg(f.avg("cosine_sim").alias("cossim")).orderBy('cossim', ascending=False).show()

+--------------------+--------------------+------------------+
|          stock_name|            combo_id|            cossim|
+--------------------+--------------------+------------------+
|10679.Asien--Aust...|06207099999_08524...|0.9680027365684509|
|10679.Asien--Aust...|03066099999_08524...|0.9667748808860779|
|10698.Asien--Aust...|06207099999_08524...|0.9648412466049194|
|10612.Asien--Aust...|06207099999_08524...| 0.964636504650116|
|10679.Asien--Aust...|03862099999_08524...|0.9641613364219666|
|1004.Asien--Austr...|06207099999_08524...|0.9641316533088684|
|10882.Asien--Aust...|06207099999_08524...|0.9635195732116699|
|10698.Asien--Aust...|03066099999_08524...|0.9632366299629211|
|1004.Asien--Austr...|03066099999_08524...|0.9627444744110107|
|10882.Asien--Aust...|03066099999_08524...|0.9626548290252686|
|10612.Asien--Aust...|03066099999_08524...|0.9624868631362915|
|10698.Asien--Aust...|03862099999_08524...| 0.961978554725647|
|10882.Asien--Aust...|03862099999_08524...|0.9614697694

In [ ]:
df_final.groupBy(["stock_name", "combo_id"]).agg(f.avg("cosine_sim").alias("cossim")).orderBy('cossim', ascending=False).count()

3800